In [1]:
import pandas as pd
import numpy as np

In [2]:
train_rides_df = pd.read_csv('DBtrainrides.csv')

In [3]:
# Split the ID column into 'ID_Base' and 'ID_Number'
train_rides_df[['ID_Base', 'ID_Timestamp', 'ID_Stop_Number']] = train_rides_df['ID'].str.rsplit('-', n=2, expand=True)

# Convert 'ID_Number' to numeric
train_rides_df['ID_Stop_Number'] = pd.to_numeric(train_rides_df['ID_Stop_Number'])

In [4]:
# Remove 'ID' column and reorder columns so 'ID_Base', 'ID_Number', 'ID_Timestamp' are at the start
new_column_order = ['ID_Base', 'ID_Timestamp', 'ID_Stop_Number'] + [col for col in train_rides_df.columns if
                                                                    col not in ['ID', 'ID_Base',
                                                                                'ID_Timestamp', 'ID_Stop_Number',
                                                                                ]
                                                                    ]
train_rides_df = train_rides_df.drop(columns=['ID'])
train_rides_df = train_rides_df[new_column_order]

In [5]:
train_rides_df_copy = train_rides_df.copy()

In [6]:
# Rename 'eva_nr' to 'starting_station_IBNR'
train_rides_df = train_rides_df.rename(columns={'eva_nr': 'starting_station_IBNR'})

# Drop starting station string, time and accompanying data 
train_rides_df.drop(['station', 'state', 'city', 'long', 'lat', 'category'], axis=1, inplace=True)
train_rides_df.drop(['arrival_plan', 'departure_plan', 'arrival_change', 'departure_change', 'arrival_delay_m', 'departure_delay_m', 'info', 'arrival_delay_check', 'departure_delay_check'], axis=1, inplace=True)

In [7]:
# Display the first few rows of the train rides DataFrame
train_rides_df.head()

,ID_Base,ID_Timestamp,ID_Stop_Number,line,path,starting_station_IBNR,zip
0,1573967790757085557,2407072312,14,20,Stolberg(Rheinl)Hbf Gl.44|Eschweiler-St.Jöris|...,8000001,52064
1,349781417030375472,2407080017,1,18,NaN,8000001,52064
2,7157250219775883918,2407072120,25,1,Hamm(Westf)Hbf|Kamen|Kamen-Methler|Dortmund-Ku...,8000406,52066
3,349781417030375472,2407080017,2,18,Aachen Hbf,8000404,52072
4,1983158592123451570,2407080010,3,33,Herzogenrath|Kohlscheid,8000404,52072


In [8]:
# Count missing values to understand data quality
train_rides_df.count()

ID_Base                  2061357
ID_Timestamp             2061357
ID_Stop_Number           2061357
line                     2061357
path                     1850002
starting_station_IBNR    2061357
zip                      2061357
dtype: int64

In [9]:
# Group by 'zip' to understand distribution
zip_df = train_rides_df.groupby('zip').size().reset_index(name='count')
zip_df

,zip,count
0,1067,2458
1,1069,2045
2,1097,3305
3,1109,1800
4,1127,597
...,...,...
1646,99817,453
1647,99867,494
1648,99880,424
1649,99947,453


In [10]:
# Step 2: Create a new DataFrame with a count of how often each 'eva_nr' is mentioned
station_mapping_df = train_rides_df['starting_station_IBNR'].value_counts().reset_index()
station_mapping_df.columns = ['starting_station_IBNR', 'count']
station_mapping_df

,starting_station_IBNR,count
0,8004128,8732
1,8089047,8312
2,8000262,7814
3,8004132,7598
4,8004131,7382
...,...,...
1991,8007768,95
1992,8005644,74
1993,8005543,49
1994,8010035,5


In [11]:
# Count number of unique 'eva_nr' to understand the spread across different stations
station_mapping_df.count()

starting_station_IBNR    1996
count                    1996
dtype: int64

In [12]:
# Step 1: Group by 'ID_Base' and 'ID_Timestamp', then find the maximum 'ID_Stop_Number'
# Use idxmax() to get the index of the row with the highest 'ID_Stop_Number' for each group
max_stop_numbers_idx = train_rides_df.groupby(['ID_Base', 'ID_Timestamp'])['ID_Stop_Number'].idxmax()

# Step 2: Filter the DataFrame to keep only the rows with the highest 'ID_Stop_Number' per group
max_stop_numbers_df = train_rides_df.loc[max_stop_numbers_idx]

# Step 3: Remove duplicates: drop rows with the same 'ID_Base' and 'ID_Timestamp'
max_stop_numbers_df = max_stop_numbers_df.drop_duplicates(subset=['ID_Base', 'ID_Timestamp'], keep='first')

max_stop_numbers_df = max_stop_numbers_df.sort_values(by=['starting_station_IBNR', 'ID_Base', 'ID_Timestamp'])

max_stop_numbers_df.head()

,ID_Base,ID_Timestamp,ID_Stop_Number,line,path,starting_station_IBNR,zip
286645,-2065137557584893414,2407082237,1,29,NaN,8000001,52064
595517,-2065137557584893414,2407092237,1,29,NaN,8000001,52064
906467,-2065137557584893414,2407102237,1,29,NaN,8000001,52064
1216357,-2065137557584893414,2407112237,1,29,NaN,8000001,52064
1523701,-2065137557584893414,2407122237,1,29,NaN,8000001,52064


In [13]:
max_stop_numbers_df.count()

ID_Base                  257818
ID_Timestamp             257818
ID_Stop_Number           257818
line                     257818
path                     217022
starting_station_IBNR    257818
zip                      257818
dtype: int64

In [14]:
# Step 4: Split the 'path' into individual stations by '|', creating a list in each row
max_stop_numbers_df = max_stop_numbers_df.assign(last_station=max_stop_numbers_df['path'].str.split('|'))

# Step 5: Explode the 'last_station' column to create individual rows for each station
exploded_stations_df = max_stop_numbers_df.explode('last_station').drop(columns=['path'])

# Step 6: Add a counter to keep track of the sequence of stops within each path
exploded_stations_df['stop_number'] = exploded_stations_df.groupby(['ID_Base', 'ID_Timestamp']).cumcount() + 1

# Step 8: Sort by 'starting_station_IBNR', 'ID_Base', 'ID_Timestamp', and then 'stop_number'
exploded_stations_df = exploded_stations_df.sort_values(by=['starting_station_IBNR', 'ID_Base', 'ID_Timestamp', 'stop_number'])

# Reset the index to keep things clean
exploded_stations_df.reset_index(drop=True, inplace=True)

# Step 9: Fill missing values in 'path' with empty strings
max_stop_numbers_df['last_station'] = max_stop_numbers_df['last_station'].replace("", np.nan)

# Display the filtered and exploded result
exploded_stations_df.head()

,ID_Base,ID_Timestamp,ID_Stop_Number,line,starting_station_IBNR,zip,last_station,stop_number
0,-2065137557584893414,2407082237,1,29,8000001,52064,NaN,1
1,-2065137557584893414,2407092237,1,29,8000001,52064,NaN,1
2,-2065137557584893414,2407102237,1,29,8000001,52064,NaN,1
3,-2065137557584893414,2407112237,1,29,8000001,52064,NaN,1
4,-2065137557584893414,2407122237,1,29,8000001,52064,NaN,1


In [15]:
exploded_stations_df

,ID_Base,ID_Timestamp,ID_Stop_Number,line,starting_station_IBNR,zip,last_station,stop_number
0,-2065137557584893414,2407082237,1,29,8000001,52064,NaN,1
1,-2065137557584893414,2407092237,1,29,8000001,52064,NaN,1
2,-2065137557584893414,2407102237,1,29,8000001,52064,NaN,1
3,-2065137557584893414,2407112237,1,29,8000001,52064,NaN,1
4,-2065137557584893414,2407122237,1,29,8000001,52064,NaN,1
...,...,...,...,...,...,...,...,...
2785184,6234297817509604666,2407112012,12,70,8098360,68642,Stockstadt(Rhein),7
2785185,6234297817509604666,2407112012,12,70,8098360,68642,Biebesheim,8
2785186,6234297817509604666,2407112012,12,70,8098360,68642,Gernsheim,9
2785187,6234297817509604666,2407112012,12,70,8098360,68642,Groß-Rohrheim,10


In [16]:
exploded_stations_df.count()

ID_Base                  2785189
ID_Timestamp             2785189
ID_Stop_Number           2785189
line                     2785189
starting_station_IBNR    2785189
zip                      2785189
last_station             2744393
stop_number              2785189
dtype: int64

In [17]:
ibnr_index_df = pd.read_csv('ibnr_stations_index.csv')

In [18]:
# Remove any leading or trailing spaces and set to lower case letters
exploded_stations_df['last_station'] = exploded_stations_df['last_station'].str.strip().str.lower()
ibnr_index_df['Station Name'] = ibnr_index_df['Station Name'].str.strip().str.lower()

In [19]:
ibnr_index_df

,IBNR,Station Name
0,8000001,aachen hbf
1,8000001,ac
2,8000001,aken c
3,8000001,aquisgrana
4,8000001,aix-la-chapelle
...,...,...
9128,8098553,hamburg-altona(s)
9129,8098555,bensersiel ne
9130,8099503,hildesheim gbf
9131,8099506,stolberg(rheinl)gbf


In [20]:
# Updated merge using 'station' instead of 'Station Name'
exploded_stations_df_with_ibnr_df = exploded_stations_df.merge(
    ibnr_index_df,
    how='left',
    left_on='last_station',
    right_on='Station Name'
)

In [37]:
exploded_stations_df_with_ibnr_df = exploded_stations_df_with_ibnr_df.drop(columns=['ID_Stop_Number'])

In [38]:
exploded_stations_df_with_ibnr_df.head(10)

,ID_Base,ID_Timestamp,line,starting_station_IBNR,zip,last_station,stop_number,IBNR,Station Name
0,-2065137557584893414,2407082237,29,8000001,52064,NaN,1,NaN,NaN
1,-2065137557584893414,2407092237,29,8000001,52064,NaN,1,NaN,NaN
2,-2065137557584893414,2407102237,29,8000001,52064,NaN,1,NaN,NaN
3,-2065137557584893414,2407112237,29,8000001,52064,NaN,1,NaN,NaN
4,-2065137557584893414,2407122237,29,8000001,52064,NaN,1,NaN,NaN
5,-2065137557584893414,2407132237,29,8000001,52064,NaN,1,NaN,NaN
6,-2065137557584893414,2407142237,29,8000001,52064,NaN,1,NaN,NaN
7,-3561454673811003901,2407082137,29,8000001,52064,NaN,1,NaN,NaN
8,-3561454673811003901,2407092137,29,8000001,52064,NaN,1,NaN,NaN
9,-3561454673811003901,2407102137,29,8000001,52064,NaN,1,NaN,NaN


In [22]:
exploded_stations_df_with_ibnr_df.columns

Index(['ID_Base', 'ID_Timestamp', 'ID_Stop_Number', 'line',
       'starting_station_IBNR', 'zip', 'last_station', 'stop_number', 'IBNR',
       'Station Name'],
      dtype='object')

In [23]:
exploded_stations_df_with_ibnr_df.count()

ID_Base                  2785189
ID_Timestamp             2785189
ID_Stop_Number           2785189
line                     2785189
starting_station_IBNR    2785189
zip                      2785189
last_station             2744393
stop_number              2785189
IBNR                     2588987
Station Name             2588987
dtype: int64

In [24]:
# Convert empty strings back to NaN in 'path' column
exploded_stations_df_with_ibnr_df['last_station'] = exploded_stations_df_with_ibnr_df['last_station'].replace('',
                                                                                                      pd.NA)

In [25]:
# Count rows where 'last_station' is NaN
nan_station_count = exploded_stations_df_with_ibnr_df['last_station'].isna().sum()
print(f"Number of rows with NaN station names: {nan_station_count}")

Number of rows with NaN station names: 40796


In [26]:
# Count where 'Station Name' is NaN and 'last_station' is not NaN
count_nan_station_name = exploded_stations_df_with_ibnr_df[
    (exploded_stations_df_with_ibnr_df['Station Name'].isna()) &
    (exploded_stations_df_with_ibnr_df['last_station'].notna())
    ].shape[0]
print(f"Number of rows with NaN IBNR that have a current station: {count_nan_station_name}")

Number of rows with NaN IBNR that have a current station: 155406


In [27]:
# Count rows where 'Station Name' is NaN
nan_station_count = exploded_stations_df_with_ibnr_df['Station Name'].isna().sum()
print(f"Number of rows with NaN station names: {nan_station_count}")

Number of rows with NaN station names: 196202


In [28]:
exploded_stations_df_with_no_ibnr_df = exploded_stations_df_with_ibnr_df[exploded_stations_df_with_ibnr_df['Station Name'].isna()]
exploded_stations_df_with_no_ibnr_df.head(10)

,ID_Base,ID_Timestamp,ID_Stop_Number,line,starting_station_IBNR,zip,last_station,stop_number,IBNR,Station Name
0,-2065137557584893414,2407082237,1,29,8000001,52064,NaN,1,NaN,NaN
1,-2065137557584893414,2407092237,1,29,8000001,52064,NaN,1,NaN,NaN
2,-2065137557584893414,2407102237,1,29,8000001,52064,NaN,1,NaN,NaN
3,-2065137557584893414,2407112237,1,29,8000001,52064,NaN,1,NaN,NaN
4,-2065137557584893414,2407122237,1,29,8000001,52064,NaN,1,NaN,NaN
5,-2065137557584893414,2407132237,1,29,8000001,52064,NaN,1,NaN,NaN
6,-2065137557584893414,2407142237,1,29,8000001,52064,NaN,1,NaN,NaN
7,-3561454673811003901,2407082137,1,29,8000001,52064,NaN,1,NaN,NaN
8,-3561454673811003901,2407092137,1,29,8000001,52064,NaN,1,NaN,NaN
9,-3561454673811003901,2407102137,1,29,8000001,52064,NaN,1,NaN,NaN


In [29]:
# TODO Find the latest version of the routes and remove all older versions

In [30]:
train_rides_df_copy.columns

Index(['ID_Base', 'ID_Timestamp', 'ID_Stop_Number', 'line', 'path', 'eva_nr',
       'category', 'station', 'state', 'city', 'zip', 'long', 'lat',
       'arrival_plan', 'departure_plan', 'arrival_change', 'departure_change',
       'arrival_delay_m', 'departure_delay_m', 'info', 'arrival_delay_check',
       'departure_delay_check'],
      dtype='object')

In [31]:
train_rides_df_copy.drop(['station', 'zip', 'state', 'city', 'category'], axis=1, inplace=True)
train_rides_df_copy.drop(['line', 'path', 'eva_nr'], axis=1, inplace=True)
train_rides_df_copy.drop(['arrival_delay_check', 'departure_delay_check'], axis=1, inplace=True)

In [32]:
train_rides_df_copy.head(20)

,ID_Base,ID_Timestamp,ID_Stop_Number,long,lat,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info
0,1573967790757085557,2407072312,14,6.091499,50.767800,2024-07-08 00:00:00,2024-07-08 00:01:00,2024-07-08 00:03:00,2024-07-08 00:04:00,3,3,NaN
1,349781417030375472,2407080017,1,6.091499,50.767800,NaN,2024-07-08 00:17:00,NaN,NaN,0,0,NaN
2,7157250219775883918,2407072120,25,6.116475,50.770202,2024-07-08 00:03:00,2024-07-08 00:04:00,2024-07-08 00:03:00,2024-07-08 00:04:00,0,0,NaN
3,349781417030375472,2407080017,2,6.070715,50.780360,2024-07-08 00:20:00,2024-07-08 00:21:00,NaN,NaN,0,0,NaN
4,1983158592123451570,2407080010,3,6.070715,50.780360,2024-07-08 00:20:00,2024-07-08 00:21:00,2024-07-08 00:20:00,2024-07-08 00:21:00,0,0,NaN
5,-5293934437045765939,2407080023,2,6.070715,50.780360,2024-07-08 00:30:00,2024-07-08 00:31:00,2024-07-08 00:30:00,2024-07-08 00:31:00,0,0,Bauarbeiten. (Quelle: zuginfo.nrw)
6,6845762881043426854,2407072357,6,6.070715,50.780360,2024-07-08 00:58:00,2024-07-08 00:58:00,NaN,NaN,0,0,NaN
7,-2100556839975301087,2407072307,13,6.070715,50.780360,2024-07-08 00:37:00,2024-07-08 00:41:00,2024-07-08 00:37:00,2024-07-08 00:41:00,0,0,NaN
8,-7696913984968518161,2407080037,1,10.096271,48.841013,NaN,2024-07-08 00:37:00,NaN,2024-07-08 00:37:00,0,0,Information
9,-6027587483204218492,2407080013,4,9.030447,53.015990,2024-07-08 00:27:00,2024-07-08 00:27:00,2024-07-08 01:16:00,2024-07-08 01:17:00,49,50,NaN


In [40]:
exploded_stations_df_with_ibnr_df.head()

,ID_Base,ID_Timestamp,line,starting_station_IBNR,zip,last_station,stop_number,IBNR,Station Name
0,-2065137557584893414,2407082237,29,8000001,52064,NaN,1,NaN,NaN
1,-2065137557584893414,2407092237,29,8000001,52064,NaN,1,NaN,NaN
2,-2065137557584893414,2407102237,29,8000001,52064,NaN,1,NaN,NaN
3,-2065137557584893414,2407112237,29,8000001,52064,NaN,1,NaN,NaN
4,-2065137557584893414,2407122237,29,8000001,52064,NaN,1,NaN,NaN


In [41]:
# Perform the merge without renaming columns, keeping all rows from exploded_stations_df_with_ibnr_df
exploded_stations_df_with_ibnr_time_df = pd.merge(
    exploded_stations_df_with_ibnr_df,
    train_rides_df_copy,
    left_on=['ID_Base', 'ID_Timestamp', 'stop_number'],  # from exploded_stations_df_with_ibnr_df
    right_on=['ID_Base', 'ID_Timestamp', 'ID_Stop_Number'],  # from train_rides_df_copy
    how='left'  # Keeps all rows from exploded_stations_df_with_ibnr_df
)

In [43]:
exploded_stations_df_with_ibnr_time_df.head(1000)

,ID_Base,ID_Timestamp,line,starting_station_IBNR,zip,last_station,stop_number,IBNR,Station Name,ID_Stop_Number,long,lat,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info
0,-2065137557584893414,2407082237,29,8000001,52064,NaN,1,NaN,NaN,1.0,6.091499,50.767800,NaN,2024-07-08 22:37:00,NaN,NaN,0.0,0.0,NaN
1,-2065137557584893414,2407092237,29,8000001,52064,NaN,1,NaN,NaN,1.0,6.091499,50.767800,NaN,2024-07-09 22:37:00,NaN,NaN,0.0,0.0,NaN
2,-2065137557584893414,2407102237,29,8000001,52064,NaN,1,NaN,NaN,1.0,6.091499,50.767800,NaN,2024-07-10 22:37:00,NaN,NaN,0.0,0.0,NaN
3,-2065137557584893414,2407112237,29,8000001,52064,NaN,1,NaN,NaN,1.0,6.091499,50.767800,NaN,2024-07-11 22:37:00,NaN,NaN,0.0,0.0,NaN
4,-2065137557584893414,2407122237,29,8000001,52064,NaN,1,NaN,NaN,1.0,6.091499,50.767800,NaN,2024-07-12 22:37:00,NaN,NaN,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,-1653513626458530012,2407101636,5,8000004,33184,weetzen,10,8000342.0,weetzen,10.0,9.635376,52.298351,2024-07-10 17:09:00,2024-07-10 17:10:00,2024-07-10 17:11:00,2024-07-10 17:12:00,2.0,2.0,Information
996,-1653513626458530012,2407101636,5,8000004,33184,holtensen/linderte,11,8002969.0,holtensen/linderte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
997,-1653513626458530012,2407101636,5,8000004,33184,bennigsen,12,8000872.0,bennigsen,12.0,9.666376,52.240733,2024-07-10 17:17:00,2024-07-10 17:17:00,2024-07-10 17:17:00,2024-07-10 17:17:00,0.0,0.0,Information
998,-1653513626458530012,2407101636,5,8000004,33184,völksen/eldagsen,13,8001741.0,völksen/eldagsen,13.0,9.626528,52.211875,2024-07-10 17:21:00,2024-07-10 17:22:00,2024-07-10 17:22:00,2024-07-10 17:23:00,1.0,1.0,Information


In [44]:
exploded_stations_df_with_ibnr_time_df.drop(['Station Name', 'ID_Stop_Number', 'ID_Stop_Number'], axis=1, inplace=True)

In [47]:
# First, sort the DataFrame by the specified columns
exploded_stations_df_with_ibnr_time_df = exploded_stations_df_with_ibnr_time_df.sort_values(
    by=['starting_station_IBNR', 'ID_Base', 'ID_Timestamp']
)

# Reorder the columns to have 'stop_number' after 'ID_Timestamp'
columns = exploded_stations_df_with_ibnr_time_df.columns.tolist()
# Move 'stop_number' after 'ID_Timestamp'
columns.remove('stop_number')
columns.insert(columns.index('ID_Timestamp') + 1, 'stop_number')
exploded_stations_df_with_ibnr_time_df = exploded_stations_df_with_ibnr_time_df[columns]

# Display the first 10 rows
exploded_stations_df_with_ibnr_time_df.head(10)

,ID_Base,ID_Timestamp,stop_number,line,starting_station_IBNR,zip,last_station,IBNR,long,lat,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info
0,-2065137557584893414,2407082237,1,29,8000001,52064,NaN,NaN,6.091499,50.7678,NaN,2024-07-08 22:37:00,NaN,NaN,0.0,0.0,NaN
1,-2065137557584893414,2407092237,1,29,8000001,52064,NaN,NaN,6.091499,50.7678,NaN,2024-07-09 22:37:00,NaN,NaN,0.0,0.0,NaN
2,-2065137557584893414,2407102237,1,29,8000001,52064,NaN,NaN,6.091499,50.7678,NaN,2024-07-10 22:37:00,NaN,NaN,0.0,0.0,NaN
3,-2065137557584893414,2407112237,1,29,8000001,52064,NaN,NaN,6.091499,50.7678,NaN,2024-07-11 22:37:00,NaN,NaN,0.0,0.0,NaN
4,-2065137557584893414,2407122237,1,29,8000001,52064,NaN,NaN,6.091499,50.7678,NaN,2024-07-12 22:37:00,NaN,NaN,0.0,0.0,NaN
5,-2065137557584893414,2407132237,1,29,8000001,52064,NaN,NaN,6.091499,50.7678,NaN,2024-07-13 22:37:00,NaN,NaN,0.0,0.0,NaN
6,-2065137557584893414,2407142237,1,29,8000001,52064,NaN,NaN,6.091499,50.7678,NaN,2024-07-14 22:37:00,NaN,NaN,0.0,0.0,NaN
7,-3561454673811003901,2407082137,1,29,8000001,52064,NaN,NaN,6.091499,50.7678,NaN,2024-07-08 21:37:00,NaN,NaN,0.0,0.0,NaN
8,-3561454673811003901,2407092137,1,29,8000001,52064,NaN,NaN,6.091499,50.7678,NaN,2024-07-09 21:37:00,NaN,NaN,0.0,0.0,NaN
9,-3561454673811003901,2407102137,1,29,8000001,52064,NaN,NaN,6.091499,50.7678,NaN,2024-07-10 21:37:00,NaN,NaN,0.0,0.0,NaN


In [48]:
exploded_stations_df_with_ibnr_time_df

,ID_Base,ID_Timestamp,stop_number,line,starting_station_IBNR,zip,last_station,IBNR,long,lat,arrival_plan,departure_plan,arrival_change,departure_change,arrival_delay_m,departure_delay_m,info
0,-2065137557584893414,2407082237,1,29,8000001,52064,NaN,NaN,6.091499,50.767800,NaN,2024-07-08 22:37:00,NaN,NaN,0.0,0.0,NaN
1,-2065137557584893414,2407092237,1,29,8000001,52064,NaN,NaN,6.091499,50.767800,NaN,2024-07-09 22:37:00,NaN,NaN,0.0,0.0,NaN
2,-2065137557584893414,2407102237,1,29,8000001,52064,NaN,NaN,6.091499,50.767800,NaN,2024-07-10 22:37:00,NaN,NaN,0.0,0.0,NaN
3,-2065137557584893414,2407112237,1,29,8000001,52064,NaN,NaN,6.091499,50.767800,NaN,2024-07-11 22:37:00,NaN,NaN,0.0,0.0,NaN
4,-2065137557584893414,2407122237,1,29,8000001,52064,NaN,NaN,6.091499,50.767800,NaN,2024-07-12 22:37:00,NaN,NaN,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2811475,6234297817509604666,2407112012,7,70,8098360,68642,stockstadt(rhein),8005740.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2811476,6234297817509604666,2407112012,8,70,8098360,68642,biebesheim,8000951.0,8.473978,49.781977,2024-07-11 20:45:00,2024-07-11 20:45:00,2024-07-11 20:45:00,2024-07-11 20:45:00,0.0,0.0,Information
2811477,6234297817509604666,2407112012,9,70,8098360,68642,gernsheim,8002249.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2811478,6234297817509604666,2407112012,10,70,8098360,68642,groß-rohrheim,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [49]:
exploded_stations_df_with_ibnr_time_df.count()

ID_Base                  2811480
ID_Timestamp             2811480
stop_number              2811480
line                     2811480
starting_station_IBNR    2811480
zip                      2811480
last_station             2770684
IBNR                     2614240
long                     1839163
lat                      1839163
arrival_plan             1627808
departure_plan           1839163
arrival_change           1388311
departure_change         1529307
arrival_delay_m          1839163
departure_delay_m        1839163
info                      591705
dtype: int64